In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
teams_dict = {
    'Real Madrid': [1, 'Spain'],
    'Manchester City': [1, 'England'],
    'Bayern Munich': [1, 'Germany'],
    'PSG': [1, 'France'],
    'Liverpool': [1, 'England'],
    'Inter Milan': [1, 'Italy'],
    'Borussia Dortmund': [1, 'Germany'],
    'RB Leipzig': [1, 'Germany'],
    'Barcelona': [1, 'Spain'],
    'Bayer Leverkusen': [2, 'Germany'],
    'Atlético Madrid': [2, 'Spain'],
    'Atalanta': [2, 'Italy'],
    'Arsenal': [2, 'England'],
    'AC Milan': [2, 'Italy'],
    'Juventus': [2, 'Italy'],
    'Benfica': [2, 'Portugal'],
    'Club Brugge KV': [2, 'Belgium'],
    'Shakhtar Donetsk': [2, 'Ukraine'],
    'Feyenoord': [3, 'Netherlands'],
    'Sporting CP': [3, 'Portugal'],
    'PSV Eindhoven': [3, 'Netherlands'],
    'Dinamo Zagreb': [3, 'Croatia'],
    'Salzburg': [3, 'Austria'],
    'Lille': [3, 'France'],
    'Crvena zvezda': [3, 'Serbia'],
    'Young Boys': [3, 'Switzerland'],
    'Celtic': [3, 'Scotland'],
    'Slovan Bratislava': [4, 'Slovakia'],
    'Monaco': [4, 'France'],
    'Sparta Prague': [4, 'Czech Republic'],
    'Aston Villa': [4, 'England'],
    'Bologna': [4, 'Italy'],
    'Girona': [4, 'Spain'],
    'Stuttgart': [4, 'Germany'],
    'Sturm Graz': [4, 'Austria'],
    'Brest': [4, 'France']
}
teams_info=pd.DataFrame.from_dict(teams_dict, orient='index', columns=['pot', 'country'])
match_table = pd.DataFrame(columns=list(teams_dict.keys()),index=list(teams_dict.keys()))


In [ ]:
# Check if the opposing countries exceed two.
def check_country_limit(country, country_count, draw_pool):
    if country not in country_count:
        country_count[country]=1
    else:
        draw_pool=draw_pool[draw_pool['country']!=country]
    return draw_pool

In [ ]:
#Eliminate the teams that have already been drawn
def remove_taken_teams(i,vs,team_pot,pot_pool,is_home):
    #check if homematch been taken
    if is_home:
        sub_vs=vs.iloc[i*9:(i+1)*9, (team_pot-1)*9:team_pot*9]
        rows_with_1 = sub_vs[(sub_vs == 1).any(axis=1)]
        indices_with_1 = rows_with_1.index.tolist()       
        drop_indices=[idx for idx in indices_with_1 if idx in pot_pool.index]
        pot_pool=pot_pool.drop(drop_indices)#exclude teams have been taken
    else:
        sub_vs=vs.iloc[i*9:(i+1)*9, (team_pot-1)*9:team_pot*9]
        rows_with_0 = sub_vs[(sub_vs == 0).any(axis=1)]
        indices_with_0 = rows_with_0.index.tolist()       
        drop_indices=[idx for idx in indices_with_0 if idx in pot_pool.index]
        pot_pool=pot_pool.drop(drop_indices)#exclude teams have been taken
    return pot_pool

In [ ]:
#Avoid a certain opponent, as there are already two teams from the same league as that opponent.
def check_opp_country(team, pot, df):
  team_country=teams_info.loc[team,'country']
  teams=pot.index.tolist()
  for i in teams:
    opp_teams = df[i].notna().index[df[i].notna()].tolist()
    opp_country_count={}
    opp_country_count[team_country]=0
    for opp_team in opp_teams: 
      opp_country=teams_info.loc[opp_team,'country']
      if opp_country not in opp_country_count:
        opp_country_count[opp_country]=1
      else:
        opp_country_count[opp_country]+=1
    if opp_country_count[team_country] ==2:
      pot=pot.drop(i)
  return pot


In [6]:
def draw_one_team(i, vs, draw_pool, team, is_home, team_pot, info, country_count):
    if is_home:
        pot_pool=draw_pool[draw_pool['pot']==i+1]

        #check if homematch been taken
        pot_pool=remove_taken_teams(i,vs,team_pot,pot_pool,True)

        # remove teams that has 2 rival in current team country
        pot_pool=check_opp_country(team,pot_pool,vs)

        # draw
        opp= random.choice(pot_pool.index)
        opp_country=info.loc[opp,'country']
        # update vs table
        vs.loc[opp,team]=1
        vs.loc[team,opp]=0

        # update pool
        draw_pool=draw_pool.drop(opp)
        #check country_number
        draw_pool=check_country_limit(opp_country, country_count, draw_pool)
    else:
        pot_pool=draw_pool[draw_pool['pot']==i+1]

        #check if awaymatch been taken
        pot_pool=remove_taken_teams(i,vs,team_pot,pot_pool,False)

        # remove teams that has 2 rival in current team country
        pot_pool=check_opp_country(team,pot_pool,vs)

        #draw
        opp= random.choice(pot_pool.index)
        opp_country=info.loc[opp,'country']

        # update vs table
        vs.loc[opp,team]=0
        vs.loc[team,opp]=1

        # update pool
        draw_pool=draw_pool.drop(opp)
        #check country_number
        draw_pool=check_country_limit(opp_country, country_count, draw_pool)
    return draw_pool,vs

In [7]:
def draw(team, vs, info, teams_info):
    team_index = info.index.get_loc(team)
    team_country = info.loc[team, 'country']
    team_pot = info.loc[team, 'pot']
    draw_pool = info.copy()
    draw_pool = draw_pool[draw_pool['country'] != team_country]  # exclude its own country, including itself
    country_count = {}
    
    drawn_club_list = list(vs[vs[team].notna()].index)
    if drawn_club_list:
        drawn_country_list = [info.loc[club, 'country'] for club in drawn_club_list]
        for country in drawn_country_list:
            draw_pool = check_country_limit(country, country_count, draw_pool)   
    
    for i in range(4):
        # two places to draw      
        if sum(vs.iloc[i*9:(i+1)*9, team_index].notna()) == 0:
            # First draw home match
            draw_pool, vs = draw_one_team(i, vs, draw_pool, team, True, team_pot, info, country_count)
            
            # Then draw away match
            draw_pool, vs = draw_one_team(i, vs, draw_pool, team, False, team_pot, info, country_count)
        
        # one place to draw
        elif sum(vs.iloc[i*9:(i+1)*9, team_index].notna()) == 1:
            t = vs.iloc[i*9:(i+1)*9, team_index].dropna()
            
            if sum(t) == 0:  # need to draw home
                draw_pool, vs = draw_one_team(i, vs, draw_pool, team, True, team_pot, info, country_count)
            
            elif sum(t) == 1:  # need to draw away
                draw_pool, vs = draw_one_team(i, vs, draw_pool, team, False, team_pot, info, country_count)
    
    return vs

In [8]:
def draw_table(teams_dict):
    vs = pd.DataFrame(columns=list(teams_dict.keys()), index=list(teams_dict.keys()))
    info = pd.DataFrame.from_dict(teams_dict, orient='index', columns=['pot', 'country'])
    teams_info = info.copy()
    
    for team in teams_dict.keys():
        vs = draw(team, vs, info, teams_info)
    
    return vs

In [9]:
def trytrytry():
    while True:
        try:
            df=draw_table(teams_dict)
            return df
        except:
            continue

In [10]:
vs_table = trytrytry()

In [11]:
vs_table.to_csv('vs_table.csv')

In [ ]:
#This rule could lead to an unsolvable situation, and official competitions don’t strictly adhere to it either.
#Avoid having two matches from the same league in either home or away fixtures."
def check_same_home_away(team,pot,df,is_home):
    if is_home:
        teams=df.index.tolist()
        for i in teams:
            if df.loc[i,team] == 1:
                # find i's country
                i_country=teams_info.loc[i,'country']
                # filter out the teams in pot from the same country
                pot = pot[pot['country'] != i_country]
    else:
        teams=df.index.tolist()
        for i in teams:
            if df.loc[i,team] == 0:
                # find i's country
                i_country=teams_info.loc[i,'country']
                # filter out the teams in pot from the same country
                pot = pot[pot['country'] != i_country]
    return pot
'''

"这个规则会导致无解，正式比赛也没有严格遵守这个规则\n#避免主场或客场出现两场比赛来自同一联赛。\ndef check_same_home_away(team,pot,df,is_home):\n    if is_home:\n        teams=df.index.tolist()\n        for i in teams:\n            if df.loc[i,team] == 1:\n                # find i's country\n                i_country=teams_info.loc[i,'country']\n                # filter out the teams in pot from the same country\n                pot = pot[pot['country'] != i_country]\n    else:\n        teams=df.index.tolist()\n        for i in teams:\n            if df.loc[i,team] == 0:\n                # find i's country\n                i_country=teams_info.loc[i,'country']\n                # filter out the teams in pot from the same country\n                pot = pot[pot['country'] != i_country]\n    return pot\n"